<a href="https://colab.research.google.com/github/littlefieldnick/2.5D-brain-mri-segmentation/blob/master/Code/FewShotSegmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install segmentation-models-pytorch grad-cam --quiet

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os

import matplotlib.pyplot as plt

from PIL import Image

import gdown

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

import albumentations as A
from albumentations.pytorch import ToTensorV2

import segmentation_models_pytorch as smp
from segmentation_models_pytorch import utils as smp_utils

from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.image import show_cam_on_image, preprocess_image


from zipfile import ZipFile

## Data Setup

In [ ]:
! wget https://github.com/pitthexai/IEEE_BHI_2023_Tutorial_From_Few_to_None/raw/bcce5fd52b349659fb03fd065f9037e70acc83a9/SampleDataset/BHI_Segmentation.zip
! unzip /content/BHI_Segmentation.zip

In [ ]:
DATA_ROOT = "/content/BHI_Segmentation"
SAVE_DIR = "/content/checkpoints/"
SHOT = 10
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
RANDOM_STATE = 42

In [ ]:
class JointSpaceSegmentationDataset(Dataset):
    def __init__(self, img_root, mask_root, image_files, mask_files, transforms=None, preprocessing=None):
        self.img_root = img_root
        self.mask_root = mask_root
        self.img_files = image_files
        self.mask_files = mask_files
        self.transforms = transforms
        self.preprocessing = preprocessing

    def __len__(self):
        return len(self.img_files)

    def __getitem__(self, idx):
        image = np.array(Image.open(os.path.join(self.img_root, self.img_files[idx])))
        mask = np.array(Image.open(os.path.join(self.mask_root, self.mask_files[idx])))

        # image = np.stack([image, image, image], axis=0)
        # mask = np.expand_dims(mask, axis=0)
        if self.transforms is not None:
            transformed = self.transforms(image=image, mask=mask)
            image = transformed["image"]
            mask = transformed["mask"]
        mask = torch.unsqueeze(mask, 0)
        if self.preprocessing is not None:
            transformed = self.preprocessing(image=image, mask=mask)
            image = transformed["image"]
            mask = transformed["mask"]

        return image.type(torch.FloatTensor), mask/255.0

In [ ]:
from sklearn.model_selection import train_test_split
def generate_datasets(root_dir):
    x_dir = os.path.join(root_dir, "Images")
    y_dir = os.path.join(root_dir, "Annotations")
    records = [[img.split(".")[0][:-1], img, img] for img in os.listdir(x_dir) if img != ".DS_Store"]

    data_records = pd.DataFrame(records, columns=["pid", "images", "masks"])

    train, test = train_test_split(data_records.pid.unique(), test_size=0.5, random_state=RANDOM_STATE)
    valid, test = train_test_split(test, test_size=0.5, random_state=RANDOM_STATE)

    train = data_records[data_records.pid.isin(train)].reset_index(drop=True)
    valid = data_records[data_records.pid.isin(valid)].reset_index(drop=True)
    test = data_records[data_records.pid.isin(test)].reset_index(drop=True)

    return train, valid, test

In [ ]:
def get_few_shot_sample(dataset, k=1, random_state=RANDOM_STATE):
    if k > len(dataset):
        return dataset

    return dataset.sample(k, random_state=random_state).reset_index(drop=True)

In [ ]:
train, valid, test = generate_datasets(os.path.join(DATA_ROOT))

In [ ]:
train_few = get_few_shot_sample(train, k=SHOT)
valid_few = get_few_shot_sample(valid, k=SHOT)

In [ ]:
train_few, valid_few

In [ ]:
def get_preprocessing_fn(encoder, encoder_weights):
    return smp.encoders.get_preprocessing_fn(encoder, encoder_weights)

def to_tensor(x, **kwargs):
    return x.transpose(2, 0, 1).astype('float32')


def get_preprocessing(preprocessing_fn):
    """Construct preprocessing transform

    Args:
        preprocessing_fn (callbale): data normalization function
            (can be specific for each pretrained neural network)
    Return:
        transform: albumentations.Compose

    """

    _transform = [
        A.Lambda(image=preprocessing_fn),
        ToTensorV2(),
    ]
    return A.Compose(_transform)

In [ ]:
if not os.path.exists(SAVE_DIR):
    os.makedirs(SAVE_DIR)

In [ ]:
train_augmentations = A.Compose([A.Resize(256, 256), A.Rotate(15), A.RandomBrightness(0.2),
                                 A.RandomContrast(0.2),  ToTensorV2()])

test_augmentations = A.Compose([A.Resize(256, 256), ToTensorV2()])
train_set = JointSpaceSegmentationDataset(os.path.join(DATA_ROOT, "Images"),
                                          os.path.join(DATA_ROOT, "Annotations"),
                                          train_few.images, train_few.masks,
                                          preprocessing=None,#get_preprocessing(preprocessing_fn),
                                          transforms=train_augmentations)

valid_set = JointSpaceSegmentationDataset(os.path.join(DATA_ROOT, "Images"),
                                          os.path.join(DATA_ROOT, "Annotations"),
                                          valid_few.images, valid_few.masks,
                                          preprocessing=None,#get_preprocessing(preprocessing_fn),
                                          transforms=test_augmentations)

test_set = JointSpaceSegmentationDataset(os.path.join(DATA_ROOT, "Images"),
                                         os.path.join(DATA_ROOT, "Annotations"),
                                         test.images, test.masks,
                                         preprocessing=None,#get_preprocessing(preprocessing_fn),
                                         transforms=test_augmentations)

## Model Training

In [ ]:
encoder = "resnet18"
encoder_weights = "imagenet"
activation = "sigmoid"
num_classes = 1 # 0=background, 1=joint space

In [ ]:
import copy
model = copy.deepcopy(smp.Unet(encoder_name=encoder, encoder_weights=encoder_weights, in_channels=1,
                 classes=num_classes, activation=activation))
model.encoder.requires_grad_ = True
model = model.to(DEVICE)
loss = nn.BCELoss()
loss.__name__="loss"
metrics = [smp_utils.metrics.IoU(threshold=0.5, activation=None), smp_utils.metrics.Fscore(0.5, activation=None)]
optimizer = torch.optim.Adam(model.parameters(), lr=5e-04)

In [ ]:
train_loader = DataLoader(train_set, batch_size=2, shuffle=True, num_workers=2)
valid_loader = DataLoader(valid_set, batch_size=2, shuffle=False, num_workers=2)

In [ ]:
# create epoch runners
# it is a simple loop of iterating over dataloader`s samples
train_epoch = smp.utils.train.TrainEpoch(
    model,
    loss=loss,
    metrics=metrics,
    optimizer=optimizer,
    device=DEVICE,
    verbose=True,
)

valid_epoch = smp.utils.train.ValidEpoch(
    model,
    loss=loss,
    metrics=metrics,
    device=DEVICE,
    verbose=True,
)

In [ ]:
# train model for 40 epochs

max_score = 0

for i in range(1, 101):

    print('\nEpoch: {}'.format(i))
    train_logs = train_epoch.run(train_loader)
    valid_logs = valid_epoch.run(valid_loader)

    # do something (save model, change lr, etc.)
    if max_score < valid_logs['iou_score']:
        max_score = valid_logs['iou_score']
        torch.save(model, f'{SAVE_DIR}/best_model_{SHOT}_shot.pth')
        print('Model saved!')


## Model Testing

In [ ]:
test_loader = DataLoader(test_set, batch_size=1, shuffle=False, num_workers=2)
test_img, test_mask  = next(iter(test_loader))

In [ ]:
model = torch.load(f'{SAVE_DIR}/best_model_{SHOT}_shot.pth')
model = model.cuda()

avg_iou = 0.0
avg_fscore = 0.0
iou_metric = smp.utils.metrics.IoU(threshold=0.5)
fscore_metric = smp.utils.metrics.Fscore(threshold=0.5)
for img, mask in test_loader:
    out = model(img.to(DEVICE))
    mask = mask.to(DEVICE)
    avg_iou += iou_metric(out, mask).item()
    avg_fscore += fscore_metric(out, mask).item()

print(f"Test IoU: {avg_iou/len(test_loader)}\nTest Dice Score: {avg_fscore/len(test_loader)}")

In [ ]:
fig, ax = plt.subplots(5, 3, figsize=(10, 10), sharex=True, sharey=True)
fig.suptitle(f"Few-Shot Segmentation Results (K = {SHOT})")
for i, (img, mask) in enumerate(test_loader):
    ax[i, 0].imshow(img[0][0,:,:].squeeze(),cmap="gray")
    ax[i, 0].set_title("X-Ray")
    ax[i, 1].imshow(mask.squeeze())
    ax[i, 1].set_title("Mask")
    ax[i, 2].imshow(torch.sigmoid(model(img.to(DEVICE)).squeeze().detach().cpu()).numpy())
    ax[i, 2].set_title("Predicted Mask")

    if (i + 1) == 5:
        break

## Comparing Different $K$-shot values

In [ ]:
gdown.download(id="1Tb3uhEfukPaFgc6KiKOpTUrpQAS1YkN8", output="/content/")

In [ ]:
!unzip /content/checkpoints.zip

In [ ]:
few_iou = []
few_dice = []

for shot in [1, 3, 5, 7, 10]:
    _, _, test = generate_datasets(os.path.join(DATA_ROOT))
    test_data = get_few_shot_sample(test, k=shot)
    test_loader = DataLoader(test_set, batch_size=1, shuffle=False, num_workers=2)

    model = torch.load(f'{SAVE_DIR}/best_model_{shot}_shot.pth')
    model = model.cuda()


    avg_iou = 0.0
    avg_fscore = 0.0
    iou_metric = smp.utils.metrics.IoU(threshold=0.5)
    fscore_metric = smp.utils.metrics.Fscore(threshold=0.5)
    for img, mask in test_loader:
        out = model(img.to(DEVICE))
        mask = mask.to(DEVICE)
        avg_iou += iou_metric(out, mask).item()
        avg_fscore += fscore_metric(out, mask).item()
    few_iou.append(avg_iou/len(test_loader))
    few_dice.append(avg_fscore/len(test_loader))

In [ ]:
plt.plot([1, 3, 5, 7, 10], few_iou)
plt.plot([1, 3, 5, 7, 10], few_dice)
plt.xlabel("K-Shot")
plt.ylabel("IoU/Dice Score")
plt.title("K-Shot IoU vs. Dice Score")
plt.legend(["IoU", "Dice Score"])

## Explainability

In [ ]:
class GradCamSampler(nn.Module):
    def __init__(self, img_root, mask_root, image_files, mask_files, transforms=None, preprocessing=None, samples_to_generate=5):
        self.img_root = img_root
        self.mask_root = mask_root
        self.img_files = image_files
        self.mask_files = mask_files
        self.samples_to_generate = samples_to_generate
        self.transforms = transforms
        self.preprocessing = preprocessing

    def _sample_images_for_gradcam(self):
        sample_ids = np.random.choice(range(len(self.img_files)), size=self.samples_to_generate)
        self.ids = sample_ids

    def __len__(self):
        return self.samples_to_generate

    def __getitem__(self, idx):
        image = np.array(Image.open(os.path.join(self.img_root, self.img_files[idx])))
        mask = np.array(Image.open(os.path.join(self.mask_root, self.mask_files[idx])))

        # image = np.stack([image, image, image], axis=-1)

        if self.transforms is not None:
            transformed = self.transforms(image=image, mask=mask)
            image = transformed["image"]
            mask = transformed["mask"]
        # mask = torch.unsqueeze(mask, 0)

        if self.preprocessing is not None:
            transformed = self.preprocessing(image=image)
            image = transformed["image"]
            # mask = transformed["mask"]

        return image.type(torch.FloatTensor), mask/255.0

In [ ]:
augmentations = A.Compose([A.Resize(256, 256), ToTensorV2()])

test_set = GradCamSampler(os.path.join(DATA_ROOT, "Images"),
                                         os.path.join(DATA_ROOT, "Annotations"),
                                         test.images, test.masks,
                                         preprocessing=None,#get_preprocessing(preprocessing_fn),
                                         transforms=test_augmentations)

In [ ]:
test_loader = DataLoader(test_set, batch_size=1, shuffle=False, num_workers=2)
test_img, test_mask  = next(iter(test_loader))

In [ ]:
class SemanticSegmentationTarget:
    def __init__(self, category, mask):
        self.category = category
        if type(mask) == np.array:
            self.mask = torch.from_numpy(mask)
        else:
            self.mask = mask

        if torch.cuda.is_available():
            self.mask = self.mask.cuda()

    def __call__(self, model_output):
        print(self.mask.shape == model_output.shape)
        return (model_output[self.category,:,:] * self.mask).sum()


In [ ]:
target_layers = [model.decoder, model.segmentation_head]
iou = smp_utils.metrics.IoU(threshold=0.5)

with GradCAM(model=model,
             target_layers=target_layers,
             use_cuda=torch.cuda.is_available()) as cam:

    fig, ax = plt.subplots(len(test_loader), 4, figsize=(10, 10), sharex=True, sharey=True)

    for i, (test_img, test_mask) in enumerate(test_loader):
        targets = [SemanticSegmentationTarget(0, test_mask)]

        grayscale_cam = cam(input_tensor=test_img,
                            targets=targets)[0, :]

        test_img2= test_img.squeeze().squeeze().detach().cpu().numpy()/255.0
        test_img2 = np.stack([test_img2, test_img2, test_img2], axis=-1)

        cam_image = show_cam_on_image(test_img2, grayscale_cam, use_rgb=True)
        out = model(test_img.to(DEVICE))

        ax[i, 0].imshow(test_img[0][0,:,:].squeeze(),cmap="gray")
        ax[i, 0].set_title("X-Ray")
        ax[i, 1].imshow(test_mask.squeeze())
        ax[i, 1].set_title("Mask")
        ax[i, 2].imshow(torch.sigmoid(out).squeeze().detach().cpu().numpy())
        ax[i, 2].set_title("Predicted Mask")
        ax[i, 3].imshow(Image.fromarray(cam_image))
        ax[i, 3].set_title("Grad-CAM")